<a href="https://colab.research.google.com/github/yoonsongg/project_EDA/blob/main/project_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#다음 분기에 어떤 게임을 설계해야 할까

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
url = '/content/drive/MyDrive/AI_Bootcamp/Section1_Project_data/vgames2.csv'

data = pd.read_csv(url, header = 0, index_col = 0)

data.head(10)

In [ ]:
#결측치 제거
data.isna().sum()
df = data.dropna()

In [ ]:
data.shape

In [ ]:
df.shape

In [ ]:
df.Genre.value_counts()
df.Genre.unique()

In [ ]:
# df.Publisher.value_counts()
# df.Publisher.unique()

In [ ]:
# df.EU_Sales.value_counts()
# df.EU_Sales.unique()

In [ ]:
# df.JP_Sales.value_counts()
# df.JP_Sales.unique()

In [ ]:
# df.Other_Sales.value_counts()
# df.Other_Sales.unique()
# df.Other_Sales.dtype

In [ ]:
df['Year'] = df['Year'].astype(int)

In [ ]:
#Year 전처리 필요
#df.Year.value_counts().tail(50)

condition  = df[df['Year'] < 100].index
condition
df.drop(condition, axis = 0, inplace = True)

In [ ]:
def str_replace(x):
    if 'M' in x:
        x = x.replace('M', '')
        return float(x)
    elif 'K' in x:
        x = x.replace('K', '')
        x = float(x)
        return round(x / 1000, 2)
    else:
        return float(x)

#type(str_replace('3.5M'))
#str_replace('310K')
df['NA_Sales'] = df['NA_Sales'].apply(lambda x : str_replace(x))
df['EU_Sales'] = df['EU_Sales'].apply(lambda x : str_replace(x))
df['JP_Sales'] = df['JP_Sales'].apply(lambda x : str_replace(x))
df['Other_Sales'] = df['Other_Sales'].apply(lambda x : str_replace(x))


In [ ]:
df.Year.describe()

In [ ]:
# Name            object
# Platform        object
# Year           float64
# Genre           object
# Publisher       object
# NA_Sales       float64
# EU_Sales       float64
# JP_Sales       float64
# Other_Sales    float64
# dtype: object

In [ ]:
#여분 만들어놓기
data3 = df
data4 = df

data4['total_Sales_notJP'] = df[['NA_Sales', 'EU_Sales', 'Other_Sales']].sum(axis = 1)


In [ ]:
#df.loc[df['Year'] == 2017]

#반드시 들어가야 하는 내용

1. 지역에 따라서 선호하는 게임 장르가 다를까

2. 연도별 게임의 트렌드가 있을까

3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스

In [ ]:
df['total_Sales'] = df[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].sum(axis = 1)

df

In [ ]:
sns.boxplot(x = 'Year', y = 'total_Sales', data = df, whis = np.inf)
sns.stripplot(x = 'Year', y = 'total_Sales', data = df, jitter = True, color = '0.4')
plt.show()

In [ ]:
q1 = df['total_Sales'].quantile(0.25)
q2 = df['total_Sales'].quantile(0.75)
iqr = q2 - q1

condition2 = ((df['total_Sales'] > q2 + 3 * iqr) | (df['total_Sales'] < q1 - 3 * iqr))
condition2 = df[condition2].index
df2 = df.drop(condition2)
df2.shape

In [ ]:
plt.figure(figsize = (20,9))
sns.boxplot(x = 'Year', y = 'total_Sales', data = df2, whis = np.inf)
sns.stripplot(x = 'Year', y = 'total_Sales', hue = 'Genre', data = df2)
plt.show()

In [ ]:
df.loc[df['Year'] == 1992].sort_values('total_Sales', ascending= False)

In [ ]:
#Genre 기준으로 total_Sales 그래프..
plt.figure(figsize = (20,9))
df2[['Genre','total_Sales']].set_index('Genre', inplace = True)
df3 = df2[['Genre', 'total_Sales']].set_index('Genre').groupby('Genre').sum('total_Sales').sort_values('total_Sales',ascending = False)


In [ ]:
df3.plot(kind = 'pie', y = 'total_Sales', autopct = '%0.2f%%')
plt.legend(df3.index, loc = 'center right', bbox_to_anchor = (1.2, 0, 0.5, 1))

# 지역에 따라 선호하는 게임 장르가 다를까


In [ ]:
#1) 데이터프레임 만들기
df_Genre_Sales = df[['Genre', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
df4 = df_Genre_Sales
df4 = df4.groupby('Genre').sum()
df4
df10 = df4.sum(axis = 0)


In [ ]:

plt.figure(figsize = (3,3))
df10.plot(kind = 'pie', autopct = '%0.2f%%')


In [ ]:
#2) 카이제곱테스트
chi_NA_EU = chi2_contingency(df4[['NA_Sales','EU_Sales']], correction = True)
chi_NA_JP = chi2_contingency(df4[['NA_Sales','JP_Sales']], correction = True)
chi_NA_Ot = chi2_contingency(df4[['NA_Sales','Other_Sales']], correction = True)
chi_EU_JP = chi2_contingency(df4[['EU_Sales','JP_Sales']], correction = True)
chi_EU_Ot = chi2_contingency(df4[['EU_Sales','Other_Sales']], correction = True)
chi_JP_Ot = chi2_contingency(df4[['JP_Sales','Other_Sales']], correction = True)

In [ ]:
chi_NA_EU # 지역과 선호하는 게임 사이 비율의 차이가 존재한다.

In [ ]:
chi_NA_JP

In [ ]:
chi_NA_Ot

In [ ]:
chi_EU_JP

In [ ]:
chi_EU_Ot #차이없음

In [ ]:
chi_JP_Ot

#연도별 게임의 트렌드가 있을까

In [ ]:
#platform 분석
# df_platform_trend = df[['Platform','Year','total_Sales']]
# df_platform_trend2 = df_platform_trend.groupby(['Year','Platform']).sum()
# df_platform_trend2 = df_platform_trend2.unstack()
# dpt = df_platform_trend2
# dpt = dpt.T.fillna(0)
# dpt.reset_index(inplace = True)
# dpt2 = dpt.iloc[:, 1:]
# dpt2.set_index('Platform', inplace = True)
# dpt_analysis = dpt2.iloc[:, 15 : 35]
# dpt_analysis = dpt_analysis.T

# plt.rcParams['figure.figsize'] = [20, 8]
# sns.lineplot(data = dpt_analysis)

# plt.show()

In [ ]:
#1) 데이터프레임 만들기

df_year_trend = df[['Genre', 'Year', 'total_Sales']]
df_year_trend

In [ ]:
df_year_trend2 = df_year_trend.groupby(['Year', 'Genre']).sum()
# df_year_trend3 = pd.DataFrame(df_year_trend2.transform('total_Sales'))
df_year_trend2 = df_year_trend2.unstack()

In [ ]:
dyt = df_year_trend2
dyt = dyt.T.fillna(0)
dyt

In [ ]:
dyt.reset_index(inplace = True)
dyt

In [ ]:
dyt2 = dyt.iloc[:, 1:]
dyt2

In [ ]:
dyt2.set_index('Genre', inplace = True)


In [ ]:
dyt2

In [ ]:
#chisquare 분석 / 의미가 있나 싶.. p-value 0.0

chi2_contingency(dyt2, correction = True)


In [ ]:
dyt_1980 = dyt2.iloc[:, 0 : 10]
dyt_1990 = dyt2.iloc[ :, 10: 20]
dyt_2000 = dyt2.iloc[:, 20 : 30]
dyt_2010 = dyt2.iloc[:, 30: ]

In [ ]:
sns.heatmap(dyt_1980, annot= True)

In [ ]:
sns.heatmap(dyt_1990, annot= True)

In [ ]:
sns.heatmap(dyt_2000, annot= True)

In [ ]:
sns.heatmap(dyt_2010, annot= True)

In [ ]:
dyt_analysis = dyt2.iloc[:, 15 : 36]
dyt_analysis = dyt_analysis.T
dyt_analysis

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
sns.lineplot(data = dyt_analysis)

plt.show()

#Action , Shooter 게임 출시

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
sns.lineplot(data = dyt_analysis[['Action','Platform','Sports','Shooter']])

plt.show()


In [ ]:
#각년도에 판매된 게임 판매량으로 게임 장르별 상관관계를 봄. -> cluster 해봄.
corr = dyt_analysis.corr()
sns.clustermap(corr)

In [ ]:
corr

# 출고량이 높은 게임에 대한 분석 및 시각화 프로세스

In [ ]:
df

In [ ]:
df.loc[df['Year'] > 2015]

In [ ]:
df2 = df[['Name', 'total_Sales']]
df_top_game = df2.groupby('Name').sum()

In [ ]:
df_top_game.sort_values('total_Sales', ascending = False, inplace = True)
df_top_game

In [ ]:
names = df_top_game.head(20)
df_top_game_info = df.loc[df['Name'].isin(names.index) == True]
names['Year'] = [2006, 2013, 1985, 1988, 2008, 2009, 1996, 2011, 2006, 2012, 2010, 2006, 2009, 1984, 2013, 1990, 2015, 2009, 2005, 2013]
names['Publisher'] = ['Nintendo', 'Take-Two Interactive', 'Nintendo', 'Nintendo', 'Nintendo','Nintendo', 'Nintendo', 'Activision','Nintendo','Activision', 'Activision', 'Nintendo', 'Nintendo', 'Nintendo','Activision', 'Nintendo', 'Activision', 'Activision', 'Nintendo','Take-Two Interactive']
names['Genre'] = ['Sports', 'Action','Platform', 'Puzzle','Racing','Sports', 'Role-Playing', 'Shooter', 'Platform','Shooter','Shooter','Misc','Platform','Shooter','Shooter','Platform','Shooter','Shooter','Simulation','Action']
names['Platform'] = ['Wii','XOne, X360, PS4, PS3, PC', 'NES,GB','NES, GB', 'Wii','Wii', 'GB', 'PC, PS3, X360, Wii', 'DS','WiiU, X360, PC, PS3', 'X369, DS, PS3, Wii','Wii','Wii','NES','PS3,X360,PS4,XOne, WiiU, PC','GBA, SNES', 'PS3, X360, XOne, PC, PS4', 'PS3, X360, PC', 'DS', 'XB, PC, X360, PS2']
df_top_game = names
df_top_game_platform = df_top_game[[ 'Year', 'Platform', 'Publisher', 'Genre', 'total_Sales']]
df_top_game_platform

In [ ]:
df_top_game_info = df_top_game_info.sort_values('total_Sales', ascending = False)

df_top_game_info.sort_values('Name')

In [ ]:
sns.pairplot(data = df_top_game)

In [ ]:
#total_Sales를 기준으로
plt.figure(figsize = (20,9))
df_top_game.plot(kind= 'pie', y = 'total_Sales', autopct = '%0.2f%%')
plt.legend(df_top_game.index, loc = 'center right', bbox_to_anchor = (1.3, 0, 0.5, 1))
df_top_game.head(10).plot(kind= 'pie', y = 'total_Sales', autopct = '%0.2f%%')
plt.legend(df_top_game.index, loc = 'center right', bbox_to_anchor = (1.3, 0, 0.5, 1))

In [ ]:
#total_sales 기준으로 publisher 파이그래프 그리기
df_top_game[['Publisher', 'total_Sales']].set_index('Publisher').groupby('Publisher').sum('total_Sales').plot(kind = 'pie', y = 'total_Sales',  autopct = '%0.2f%%')

In [ ]:
#Genre 기준으로
df_top_game[['Genre','total_Sales']].set_index('Genre').groupby('Genre').sum('total_Sales').plot(kind = 'pie', y = 'total_Sales', autopct = '%0.2f%%')
plt.legend(df_top_game.Genre, loc = 'center right', bbox_to_anchor = (1, 0, 0.5, 1))

#일본 포함하지 않는다면

In [ ]:
data4

In [ ]:
df_year_trend_notJP = data4[['Genre', 'Year', 'total_Sales_notJP']]
df_year_trend_notJP

In [ ]:
df_year_trend_notJP = df_year_trend_notJP.groupby(['Year', 'Genre']).sum()
# df_year_trend3 = pd.DataFrame(df_year_trend2.transform('total_Sales'))
df_year_trend_notJP = df_year_trend_notJP.unstack()


In [ ]:
dyt_notJP = df_year_trend_notJP
dyt_notJP = dyt_notJP.T.fillna(0)
dyt_notJP.reset_index(inplace = True)

In [ ]:
dyt_notJP = dyt_notJP.iloc[:, 1:]

In [ ]:
dyt_notJP.set_index('Genre', inplace = True)
dyt_notJP

In [ ]:
dyt_analysis_notJP = dyt_notJP.iloc[:, 15 : 36]
dyt_analysis_notJP = dyt_analysis_notJP.T
dyt_analysis_notJP

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
sns.lineplot(data = dyt_analysis_notJP)

plt.show()

#Action , Shooter 게임 출시

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
sns.lineplot(data = dyt_analysis_notJP[['Action','Platform','Sports','Shooter']])

plt.show()


In [ ]:
df_notJP = data4[['Name', 'total_Sales_notJP']]
df_top_game_notJP = df_notJP.groupby('Name').sum()

In [ ]:
df_top_game_notJP.sort_values('total_Sales_notJP', ascending = False, inplace = True)
name_notJP = df_top_game_notJP.head(20)
name_notJP['Genre'] = ['Sports', 'Action','Platform', 'Racing', 'Shooter','Puzzle','Sports','Shooter','Shooter','Shooter','Shooter','Misc','Shooter','Shooter','Platform','Platform','Action','Simulation','Misc','Platform']
name_notJP = pd.DataFrame(name_notJP)
name_notJP

In [ ]:
# df_top_game_info_notJP = df.loc[df['Name'].isin(name_notJP.index) == True]
# df_top_game_info_notJP

In [ ]:
#일본 제외 top20개의 게임에 대한 장르별 비율
name_notJP[['Genre','total_Sales_notJP']].set_index('Genre').groupby('Genre').sum('total_Sales_notJP').plot(kind = 'pie', y = 'total_Sales_notJP', autopct = '%0.2f%%')

plt.legend(name_notJP.Genre, loc = 'center right', bbox_to_anchor = (1, 0, 0.5, 1))

In [ ]:
#일본제외 전체 판매량에 대한 비율
data5 = data4[['Genre', 'total_Sales_notJP']].set_index('Genre').groupby('Genre').sum('total_Sales_notJP').sort_values('total_Sales_notJP',ascending = False)
data5.plot(kind = 'pie', y = 'total_Sales_notJP', autopct = '%0.2f%%')
plt.legend(data5.index, loc = 'center right', bbox_to_anchor = (0.9, 0, 0.5, 1))

#onlyJP

In [ ]:
df_year_trend_onlyJP = data4[['Genre', 'Year', 'JP_Sales']]
df_year_trend_onlyJP

In [ ]:
df_year_trend_onlyJP = df_year_trend_onlyJP.groupby(['Year', 'Genre']).sum()
# df_year_trend3 = pd.DataFrame(df_year_trend2.transform('total_Sales'))
df_year_trend_onlyJP = df_year_trend_onlyJP.unstack()

In [ ]:
dyt_onlyJP = df_year_trend_onlyJP
dyt_onlyJP = dyt_onlyJP.T.fillna(0)
dyt_onlyJP.reset_index(inplace = True)

In [ ]:
dyt_onlyJP = dyt_onlyJP.iloc[:, 1:]

In [ ]:
dyt_onlyJP.set_index('Genre', inplace = True)
dyt_onlyJP

In [ ]:
dyt_analysis_onlyJP = dyt_onlyJP.iloc[:, 15 : 36]
dyt_analysis_onlyJP = dyt_analysis_onlyJP.T
dyt_analysis_onlyJP

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
sns.lineplot(data = dyt_analysis_onlyJP[['Action','Platform','Sports','Role-Playing']])

plt.show()


In [ ]:
corrJP = dyt_analysis_onlyJP.corr()
sns.clustermap(corrJP)

In [ ]:
df3 = df[['Name', 'JP_Sales']]
df_top_game_onlyJP = df3.groupby('Name').sum()
df_top_game_onlyJP.sort_values('JP_Sales', ascending = False, inplace = True)
df_top_game_onlyJP

In [ ]:
names_onlyJP = df_top_game_onlyJP.head(20)
df_top_game_onlyJP_info = df.loc[df['Name'].isin(names_onlyJP.index) == True]
names_onlyJP['Genre'] = ['Role-Playing','Role-Playing','Platform','Platform','Role-Playing','Puzzle','Role-Playing','Role-Playing','Simulation','Puzzle','Role-Playing','Role-Playing','Platform','Platform', 'Role-Playing','Platform','Role-Playing','Role-Playing','Simulation','Role-Playing']

In [ ]:
names_onlyJP

In [ ]:
df_top_game_onlyJP_info.sort_values('Name')

In [ ]:
#2015년 일본에서 Action 게임 급 흥행 탑 10
df_JP_Action = df.loc[(df['Year'] == 2015) & (df['Genre'] == 'Action')]
df_JP_Action.groupby('Name').sum('JP_Sales').sort_values('JP_Sales', ascending = False).head(10)

In [ ]:
#top20 게임에 대한 파이그래프
names_onlyJP[['Genre','JP_Sales']].set_index('Genre').groupby('Genre').sum('JP_Sales').plot(kind = 'pie', y = 'JP_Sales', autopct = '%0.2f%%')

plt.legend(names_onlyJP.index, loc = 'center right', bbox_to_anchor = (1, 0, 0.5, 1))

In [ ]:
data6 = data4[['Genre', 'JP_Sales']].set_index('Genre').groupby('Genre').sum('JP_Sales').sort_values('JP_Sales',ascending = False)
data6.plot(kind = 'pie', y = 'JP_Sales', autopct = '%0.2f%%')
plt.legend(data6.index, loc = 'center right', bbox_to_anchor = (0.9, 0, 0.5, 1))

#연습

In [ ]:
# def str_replace(x):
#     x = x.replace('M', '')
#     x = x.replace('K', '')
#     return x
# #M, K 들어가 있는 거 말고 다른 값에 대해서 else: 로 빼둬야됨....ㅠ 개삽질.. ㅠ

# s1 = str_replace('3.5K')
# type(s1)